In [1]:
import numpy as np # linear algebra
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import cv2 as cv
import os

In [2]:
root_dir = "skin-cancer-mnist-ham10000"


In [16]:
skin_cancer_cells = pd.read_csv(root_dir + "/HAM10000_metadata.csv")
print(skin_cancer_cells)
all_ages = list(skin_cancer_cells.age)
is_valid = lambda x: not(np.isnan(x) or not x)
all_ages = list(filter(is_valid, all_ages))
avg = int(sum(all_ages)/len(all_ages))

for ind, age in enumerate(skin_cancer_cells['age']):
    if not is_valid(age):
        skin_cancer_cells.loc[ind, 'age'] = avg


         lesion_id      image_id     dx dx_type   age     sex localization
0      HAM_0000118  ISIC_0027419    bkl   histo  80.0    male        scalp
1      HAM_0000118  ISIC_0025030    bkl   histo  80.0    male        scalp
2      HAM_0002730  ISIC_0026769    bkl   histo  80.0    male        scalp
3      HAM_0002730  ISIC_0025661    bkl   histo  80.0    male        scalp
4      HAM_0001466  ISIC_0031633    bkl   histo  75.0    male          ear
...            ...           ...    ...     ...   ...     ...          ...
10010  HAM_0002867  ISIC_0033084  akiec   histo  40.0    male      abdomen
10011  HAM_0002867  ISIC_0033550  akiec   histo  40.0    male      abdomen
10012  HAM_0002867  ISIC_0033536  akiec   histo  40.0    male      abdomen
10013  HAM_0000239  ISIC_0032854  akiec   histo  80.0    male         face
10014  HAM_0003521  ISIC_0032258    mel   histo  70.0  female         back

[10015 rows x 7 columns]
['scalp' 'ear' 'face' 'back' 'trunk' 'chest' 'upper extremity' 'abdomen'
 

In [26]:
localizations = list(skin_cancer_cells.localization.unique())
sexes = list(skin_cancer_cells.sex.unique())

for ind, loc in enumerate(skin_cancer_cells['localization']):
    skin_cancer_cells.loc[ind, "localization_index"] = localizations.index(loc)

for ind, sex in enumerate(skin_cancer_cells['sex']):
    skin_cancer_cells.loc[ind, "sex_index"] = sexes.index(sex)



In [30]:
print(skin_cancer_cells)

         lesion_id      image_id     dx dx_type   age     sex localization  \
0      HAM_0000118  ISIC_0027419    bkl   histo  80.0    male        scalp   
1      HAM_0000118  ISIC_0025030    bkl   histo  80.0    male        scalp   
2      HAM_0002730  ISIC_0026769    bkl   histo  80.0    male        scalp   
3      HAM_0002730  ISIC_0025661    bkl   histo  80.0    male        scalp   
4      HAM_0001466  ISIC_0031633    bkl   histo  75.0    male          ear   
...            ...           ...    ...     ...   ...     ...          ...   
10010  HAM_0002867  ISIC_0033084  akiec   histo  40.0    male      abdomen   
10011  HAM_0002867  ISIC_0033550  akiec   histo  40.0    male      abdomen   
10012  HAM_0002867  ISIC_0033536  akiec   histo  40.0    male      abdomen   
10013  HAM_0000239  ISIC_0032854  akiec   histo  80.0    male         face   
10014  HAM_0003521  ISIC_0032258    mel   histo  70.0  female         back   

       localization_index  sex_index  
0                     0.